<a href="https://colab.research.google.com/github/jdl20515/retinal-mobilenetv2-model/blob/main/Retinal_MobileNetV2_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import os
import tensorflow as tf
import zipfile

from tensorflow.keras import Model, Input
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Conv2D, Flatten, MaxPooling2D, BatchNormalization
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image
%matplotlib inline


#os.environ['KAGGLE_CONFIG_DIR'] = "/content"

In [ ]:
#!kaggle datasets download -d kssanjaynithish03/retinal fundus imagesi

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
dataset_path = '/content/drive/MyDrive/Retinal_Fundus_Images'

In [ ]:
# Make training and valdiation sets
train = ImageDataGenerator(rescale = 1/255)
val = ImageDataGenerator(rescale = 1/255)

training_set = train.flow_from_directory("/content/drive/MyDrive/Retinal_Fundus_Images/train",
                                         target_size = (224,224),
                                         batch_size = 16,
                                         class_mode = "categorical"
                                         )

validation_set = train.flow_from_directory("/content/drive/MyDrive/Retinal_Fundus_Images/val",
                                         target_size = (224,224),
                                         batch_size = 16,
                                         class_mode = "categorical"
                                         )

testing_set = train.flow_from_directory("/content/drive/MyDrive/Retinal_Fundus_Images/test")

In [ ]:
# Make Model
base_model = MobileNetV2(weights = "imagenet", include_top=False, input_shape=(224,224, 3))
x = base_model.output


# Convolutional and Pooling Layers
x = Conv2D(64, (3, 3), activation="relu", input_shape = (224, 224, 3), padding="same", strides=1)(x)
x = MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding='same')(x)
x = BatchNormalization()(x)

x = Conv2D(64, (3, 3), activation="relu", padding="same", strides=1)(x)
x = MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding='same')(x)
x = BatchNormalization()(x)

x = Conv2D(128, (3, 3), activation="relu", padding="same", strides=1)(x)
x = MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding='same')(x)
x = BatchNormalization()(x)

x = Conv2D(128, (3, 3), activation="relu", padding="same", strides=1)(x)
x = MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding='same')(x)
x = BatchNormalization()(x)

# Flatten and Dropout
x = Flatten()(x)

# Normal Layers With Dropout
x = Dense(128, activation="relu")(x)
x = Dropout(0.01)(x)
x = Dense(128, activation="relu")(x)
x = Dropout(0.01)(x)
x = Dense(128, activation="relu")(x)
x = Dropout(0.01)(x)
x = Dense(128, activation="relu")(x)
x = Dropout(0.01)(x)

preds = Dense(11, activation="softmax")(x)

model = Model(inputs = base_model.input, outputs = preds)

In [ ]:
# Train and run model
epochs = 20
optimizer = Adam(learning_rate=0.0001)
model.compile(loss = "categorical_crossentropy", optimizer = optimizer, metrics = ["Accuracy"])
model.fit(training_set, validation_data = validation_set, epochs = epochs)

NameError: ignored